In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import json
import sys
import torch

sys.path.append("../")
from environments.sac_tracking_env import Environment
from models import ResNet, ResidualBlock, ConvNet
from solvers import sac

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float32

## Instantiate environment

In [5]:
config_file = "/Users/brysongray/git_repos/tractography/configs/sac_inference_simulated.json"
with open(config_file) as f:
    params = json.load(f)

img_path = params["img_path"]
outdir = params["outdir"]
name = params["name"]
step_size = params["step_size"] if "step_size" in params else 1.0
step_width = params["step_width"] if "step_width" in params else 1.0
alpha = params["alpha"] if "alpha" in params else 1.0
beta = params["beta"] if "beta" in params else 1e-3
friction = params["friction"] if "friction" in params else 1e-4
patch_radius = 17

if "classifier_weights" in params:
    classifier_path = params["classifier_weights"]
    classifier_state_dict = torch.load(classifier_path, weights_only=True)
    classifier = ResNet(ResidualBlock, [3, 4, 6, 3], num_classes=1)
    classifier = classifier.to(device=DEVICE, dtype=dtype)
    classifier.load_state_dict(classifier_state_dict)
    classifier.eval()
else:
    classifier = None

env = Environment(img_path,
                radius=patch_radius,
                step_size=step_size,
                step_width=step_width,
                max_len=10000,
                alpha=alpha,
                beta=beta,
                friction=friction,
                classifier=classifier)

UnpicklingError: Weights only load failed. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution.Do it only if you get the file from a trusted source. WeightsUnpickler error: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## Instantiate actor network

In [11]:
in_channels = 4
actor = ConvNet(chin=in_channels, chout=4)
actor = actor.to(device=DEVICE,dtype=dtype)
if "sac_weights" in params:
    sac_weights_path = params["sac_weights"]
    sac_state_dicts = torch.load(sac_weights_path, weights_only=True)

UnpicklingError: Weights only load failed. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution.Do it only if you get the file from a trusted source. WeightsUnpickler error: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
sac.inference(env, actor, outdir)